# 🧨 Diffusers meets Video

This colab showcases the new research text-to-video model by Alibaba and its integration with the diffusers library https://huggingface.co/damo-vilab/text-to-video-ms-1.7b 

In [1]:
#@title Check your GPU!
!nvidia-smi

Sat Mar 25 22:43:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Kode diatas memeriksa spesifikasi GPU yang tersedia pada mesin yang digunakan.

'#@title Check your GPU!' digunakan untuk memberikan judul atau nama pada input dialog ketika kode dijalankan.

'!nvidia-smi' merupakan perintah command line pada sistem operasi Linux yang menampilkan informasi tentang GPU Nvidia yang terpasang pada mesin. Kode ini akan menampilkan informasi spesifikasi GPU yang terdapat pada mesin seperti nama GPU, versi driver, versi CUDA, suhu, konsumsi daya, dan penggunaan memori.

Hasil keluaran yang diperoleh dari kode tersebut adalah informasi spesifikasi GPU yang terdapat pada mesin yang digunakan. Pada contoh hasil keluaran yang ditampilkan, terdapat informasi mengenai GPU Nvidia Tesla T4, suhu GPU, penggunaan daya, dan penggunaan memori pada GPU. Tidak ada proses yang sedang berjalan pada GPU pada saat kode dijalankan.

In [2]:
#@title Install dependencies
!pip install torch==2.0.0 git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-mk9_w3bo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-mk9_w3bo
  Resolved https://github.com/huggingface/diffusers to commit 9fb02175485db873664cd5841c72add6ac512692
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63

Kode diatas adalah kode untuk meng-install beberapa dependencies yaitu 'torch', 'diffusers', 'transformers', 'accelerate', dan 'imageio[ffmpeg]'.

torch==2.0.0 adalah library machine learning PyTorch versi 2.0.0
git+https://github.com/huggingface/diffusers adalah library Diffuser yang merupakan bagian dari Hugging Face Transformers, yaitu library NLP yang sangat populer.

transformers adalah library NLP yang sangat populer.
accelerate adalah library Deep Learning yang membantu mempercepat training dan inference pada GPU dan CPU dengan pengaturan konfigurasi otomatis.
imageio[ffmpeg] adalah library untuk membaca, menulis dan memproses gambar dan video di Python dengan dukungan untuk format gambar dan video yang berbeda termasuk video dengan codec FFmpeg.

In [3]:
#@title Setup pipeline
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

/usr/local/lib/python3.9/dist-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Kode di atas melakukan beberapa hal sebagai berikut:

* Mengimpor library dan modul yang diperlukan seperti torch, diffusers, IPython, dan base64.
* Membuat instance dari kelas DiffusionPipeline yang diinisialisasi dengan model pre-trained "damo-vilab/text-to-video-ms-1.7b", tipe data torch.float16, dan variasi "fp16".
* Mengubah scheduler pada pipeline menjadi instance dari kelas DPMSolverMultistepScheduler dari konfigurasi pipeline.scheduler.
Mengaktifkan offload CPU pada model pada pipeline dengan menggunakan method 'enable_model_cpu_offload()'.
* Mengaktifkan VAE slicing pada pipeline dengan menggunakan method 'enable_vae_slicing()'.

Tujuan dari setup pipeline ini adalah untuk membuat pipeline yang siap untuk digunakan pada tugas pembuatan video dari teks menggunakan teknik diffusion models. Pipeline ini telah diatur dengan parameter yang diperlukan untuk menjalankan model dengan baik dan efisien.

In [17]:
#@title Generate your video
prompt = 'Spiderman ' #@param {type:"string"}
video_duration_seconds = 3 #@param {type:"integer"}
num_frames = video_duration_seconds * 10
video_frames = pipe(prompt, negative_prompt="low quality", num_inference_steps=25, num_frames=num_frames).frames
video_path = export_to_video(video_frames)

  0%|          | 0/25 [00:00<?, ?it/s]

Kode di atas melakukan beberapa hal sebagai berikut:

* Mendefinisikan prompt yang akan digunakan untuk membuat video dengan variabel prompt.
* Menentukan durasi video dalam detik dengan variabel video_duration_seconds.
* Menghitung jumlah frame yang akan digunakan untuk membuat video dengan mengalikan durasi video dengan 10 frame per detik menggunakan variabel num_frames.
* Membuat video frames dengan memanggil instance dari kelas DiffusionPipeline (pipe) dan memberikan prompt dan parameter lainnya seperti negative_prompt, num_inference_steps, dan num_frames.
* Mengekspor video frames menjadi file video dengan memanggil fungsi export_to_video dan menyimpannya di dalam variabel video_path.
Dengan demikian, kode di atas digunakan untuk menghasilkan video dengan menggunakan model yang telah dibuat pada kode sebelumnya. Video yang dihasilkan didasarkan pada prompt yang ditentukan dan menggunakan parameter lainnya seperti negative_prompt dan num_inference_steps untuk memperbaiki kualitas video yang dihasilkan. Video yang dihasilkan kemudian diekspor menjadi file video dengan menggunakan fungsi export_to_video.

In [18]:
#@title Display the video
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video = imageio.mimread(video_path)  #Loading video
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5